# TPS_Oct_21_LGB

# Sorry but that... I find a big mistake. so I fix it. Let's be careful to use cut & paste. 😅  Thank you!

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# try faster
import datatable as dt
train = dt.fread(r'/kaggle/input/tabular-playground-series-oct-2021/train.csv').to_pandas()
test = dt.fread(r'/kaggle/input/tabular-playground-series-oct-2021/test.csv').to_pandas()

* reference:http://www.kaggle.com/m1y7k8/tps-oct-21-eda 👀

In [ ]:
# do not convert int8 ! 
id = test['id'].copy()
print(id[0:5])

In [ ]:
# Reduce memory usage

# train features
for col in train.columns:
    if train[col].dtype == "float64":
        train[col]=pd.to_numeric(train[col], downcast="float")
    if train[col].dtype == "int64":
        train[col]=pd.to_numeric(train[col], downcast="integer")
    

# test features
for col in test.columns:
    if test[col].dtype == "float64":
        test[col]=pd.to_numeric(test[col], downcast="float")
    if train[col].dtype == "int64":
        test[col]=pd.to_numeric(test[col], downcast="integer")  
        
train.info(), test.info()

In [ ]:
# bool to int8
col_1 = ['f' + str(i) for i in range(242, 285)] + ['f22','f43'] + ['target']
train[col_1] = train[col_1].astype('int8')


col_2 = ['f' + str(i) for i in range(242, 285)] + ['f22','f43'] 
test[col_2] = test[col_2].astype('int8')

train.info(), test.info()

In [ ]:
train = train.drop('id' ,axis = 1)
test = test.drop('id', axis = 1)

In [ ]:
train.head(), test.head()

---------------------

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import GradientBoostingClassifier # boost for lGB

from lightgbm import LGBMClassifier
import lightgbm as lgb

from sklearn.metrics import roc_curve, auc


In [ ]:
X = train.drop('target' , axis = 1).copy()
y = train['target'].copy()
X_test = test.copy()

# memory release
del train
del test

In [ ]:
# add features


X['std'] = X.std(axis = 1)
X['min'] = X.min(axis = 1)
X['max'] = X.max(axis = 1)


X_test['std'] = X_test.std(axis = 1)
X_test['min'] = X_test.min(axis = 1)
X_test['max'] = X_test.max(axis = 1)

-------------

In [ ]:
#scaler
from sklearn.preprocessing import QuantileTransformer

quantile = QuantileTransformer()
X = pd.DataFrame(quantile.fit_transform(X))
X_test = pd.DataFrame(quantile.transform(X_test))

#default n_quantiles = 1000,random_state = None

* Tips: 
* using scaler is better than nothing


In [ ]:
# default parameters
param = {
        'objective' : 'binary',
        'metric' : 'auc',
        'verbosity': -1,
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
         'boosting_type': 'gbdt',
        'device' : 'gpu',
        'feature_pre_filter': False, 
   
}


* to find the best param then get a good score but... it is tough
* Good Luck!

In [ ]:
# time counter ⏳
import time
start = time.perf_counter()
print("--------start----------")


In [ ]:
splits = 5
seed = 2021
skf = StratifiedKFold(n_splits = splits, shuffle=True, random_state=seed)

scores = []
pred = []

for fold, (idx_train, idx_valid) in enumerate(skf.split(X, y)):
    
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    model = LGBMClassifier(**param)                     
    model.fit( X_train, y_train,  eval_set = [(X_train,y_train),(X_valid, y_valid)], verbose=False)   

    pred_valid = model.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)

    test_preds = model.predict_proba(X_test)[:,1]
    pred.append(test_preds)
    
    print("fold : ", fold , "   LGB score : ", score)
    
print(scores)

In [ ]:
# done ⌛
time_end = time.perf_counter()
process_time = time_end - start
print("process_time:{0}".format(process_time) )

In [ ]:
predictions = np.mean(np.column_stack(pred), axis = 1)

In [ ]:

submission = pd.DataFrame({'id' : id, 'target' : predictions})
submission.to_csv("submission.csv", index = False)

submission

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.kdeplot(submission['target'],fill=True,  color = "#20B2AA")